# Imports

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import pydicom
import cv2

from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms
from torchinfo import summary

# EDA

In [ ]:
df_test = pd.read_csv('/kaggle/input/rsna-breast-cancer-detection/test.csv')
df_train = pd.read_csv('/kaggle/input/rsna-breast-cancer-detection/train.csv')

In [ ]:
df_train.head()

In [ ]:
df_train.isna().sum()

In [ ]:
df_train.drop(['BIRADS', 'density'], axis=1, inplace=True)

In [ ]:
df_train[df_train['age'].isna()] = int(df_train['age'].mean())

In [ ]:
df_train.isna().sum()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10,4))
sns.histplot(df_train, x='cancer',  bins=2, ax=axes[0])
axes[0].set_xticks([0, 1])

res_counts = df_train['cancer'].value_counts()
axes[1] = plt.pie(res_counts, labels=res_counts.index, autopct='%1.1f%%')
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 3))

sns.kdeplot(data=df_train, x='age', hue='cancer', fill=True, common_norm=False, ax=axes[0])
axes[0].set_title('Age vs. Cancer')

no_cancer = df_train[df_train['cancer'] == 0]['age']
cancer_with_invasive = df_train[(df_train['cancer'] == 1) & (df_train['invasive'] == 1)]['age']
cancer_no_invasive = df_train[(df_train['cancer'] == 1) & (df_train['invasive'] == 0)]['age']

sns.kdeplot(no_cancer, label='No Cancer', fill=True, ax=axes[1])
sns.kdeplot(cancer_with_invasive, label='Cancer with Invasive', fill=True, ax=axes[1])
sns.kdeplot(cancer_no_invasive, label='Cancer without Invasive', fill=True, ax=axes[1])

axes[1].set_title('Age vs cancer with invasiveness')
axes[1].set_xlabel('Age')
axes[1].set_ylabel('Density')
legend = axes[1].legend()
legend.set_title('Conditions')

plt.show

In [ ]:
dicom_file_path = "/kaggle/input/rsna-breast-cancer-detection/train_images/10006/462822612.dcm"
ds = pydicom.dcmread(dicom_file_path)
print(ds)

In [ ]:
if 'PixelData' in ds:
    import matplotlib.pyplot as plt
    plt.imshow(ds.pixel_array, cmap='gray')
    plt.show()
else:
    print("This DICOM file does not contain any images.")

In [ ]:
image_data = ds.pixel_array

fig, axes = plt.subplots(1, 2, figsize=(7,7))

cropped_image = image_data[:, :int(ds.WindowCenter[0])]
axes[0].imshow(cropped_image, cmap='gray')
axes[0].set_xlabel(f'ds.WindowCenter[0]')

# Crop the image using the line position
cropped_image = image_data[:, :int(ds.WindowCenter[3])]
axes[1].imshow(cropped_image, cmap='gray')
axes[1].set_xlabel(f'ds.WindowCenter[3]')

plt.show()

In [ ]:
dicom_file_path = "/kaggle/input/rsna-breast-cancer-detection/train_images/10006/1874946579.dcm"
ds = pydicom.dcmread(dicom_file_path)
print(ds)

In [ ]:
int(ds.WindowCenter[3]/10)

In [ ]:
img = cv2.imread('/kaggle/input/rsna-breast-cancer-256-pngs/10006_1459541791.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# plt.imshow(img[:, :int(ds.WindowCenter[3]/18)], cmap='gray')
plt.imshow(img, cmap='gray')

# Downsample

In [ ]:
df_target1 = df_train[df_train['cancer'] == 1]
df_target0 = df_train[df_train['cancer'] == 0]

n_idx0 = len(df_target0)
n_idx1 = len(df_target1)

df_target0_downsampled = df_target0.sample(n=n_idx1, random_state=42)
df_downsampled = pd.concat([df_target0_downsampled, df_target1])
df_downsampled = df_downsampled.sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
print(len(df_downsampled[df_downsampled['cancer'] == 0]))
print(len(df_downsampled[df_downsampled['cancer'] == 1]))

In [ ]:
from sklearn.model_selection import train_test_split

df_train_downsampled, df_val_downsampled = train_test_split(df_downsampled, test_size=0.2, random_state=42)

In [ ]:
class BreastDataset(Dataset):
    def __init__(self, df, img_size, image_dir, transform=None, additional_transforms=None):
        super(BreastDataset, self).__init__()
        
        self.df = df
        self.img_size = img_size
        self.image_dir = image_dir
        self.transform = transform
        self.additional_transforms = additional_transforms
        
    def __len__(self):
        return len(self.df)
    
    def  __getitem__(self, idx):
        row = self.df.iloc[idx]
        label = self.df.iloc[idx]['cancer']
        file_name = str(row['patient_id']) + "_" + str(row['image_id']) + ".png"
        image_path = self.image_dir + file_name
        
        image = Image.open(image_path).convert('RGB')
#         image = cv2.imread(image_path)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            image = transform(image)
        
        if label == 1 and self.additional_transforms:
            image = additional_transform(image)
            
        return image, label
    

In [ ]:
img_path_256 = '/kaggle/input/rsna-breast-cancer-512-pngs/'
image_size = (512, 512)
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
additional_transform = transforms.RandomApply([
    transforms.RandomRotation(degrees=(-20, 20))],
    p=0.5)

ds_train = BreastDataset(df_train_downsampled, image_size, img_path_256, transform, additional_transform)
ds_val = BreastDataset(df_val_downsampled, image_size, img_path_256, transform, None)

In [ ]:
from torch.utils.data import DataLoader

BATCH_SIZE = 32
train_loader = DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(ds_val, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
# plt.imshow(ds_train[0][0].numpy().transpose(1,2,0), cmap='gray')

# Model Building

### CNN

In [ ]:
IMAGE_WIDTH     = image_size[0]
IMAGE_HEIGHT    = image_size[1]
IMAGE_CHANNELS  = 3
DEVICE          = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def calc_maxpool(size, stride, kernel_size):
    return (size - kernel_size) // stride + 1

In [ ]:
calc_maxpool(254, 2, 2) # 3
calc_maxpool(125, 2, 2) # 32
calc_maxpool(60, 2, 2) # 64
calc_maxpool(28, 2, 2)  # 128
calc_maxpool(12, 2, 2) # 256
calc_maxpool(10, 2, 2) # 512
calc_maxpool(3, 2, 1) # 512

In [ ]:
def calculate_output_size_with_dilation(input_size, kernel_size, padding, stride=1, dilation=1):
    return (input_size + 2 * padding - dilation * (kernel_size - 1) - 1) // stride + 1
#     return (input_size - kernel_size + (kernel_size - 1) * (dilation - 1) + 2 * padding) // stride + 1

In [ ]:
calculate_output_size_with_dilation(input_size=256, kernel_size=3, padding=0, stride=1, dilation=1)
calculate_output_size_with_dilation(input_size=512, kernel_size=3, padding=0, stride=1, dilation=1)

In [ ]:
calculate_output_size_with_dilation(input_size=256, kernel_size=3, padding=1, stride=1, dilation=2)

In [ ]:
class CNN(nn.Module):
    def __init__(self, in_channels=[3, 32, 64, 128, 256, 512, 512]):
        super(CNN, self).__init__()
        
        self.pool = nn.MaxPool2d(2, 2)
        self.last_pool = nn.MaxPool2d(2, 1)
        self.blocks = nn.ModuleList()
        for idx, i in enumerate(in_channels):
            if i != in_channels[-1]:
                out_channels = in_channels[idx+1]
                pool = self.pool
            else:
                out_channels = in_channels[-1]
                pool = None
            
            self.blocks.append(ConvStridedBlock(i, out_channels, 3, 0, 1, 2))
            if pool:
                self.blocks.append(pool)
        
        self.fc1 = nn.Linear(512 * 10 * 10, 4096)
        self.fc2 = nn.Linear(4096, 2048)
        self.fc3 = nn.Linear(2048, 1)
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        batch_size = x.size(0)
        for block in self.blocks:
            x = block(x)
            
        x = x.view(batch_size, -1)
        
        x = self.dropout(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return self.sigmoid(x)
    
class ConvStridedBlock(nn.Module):
    def __init__(self, input_size, out_channels, kernel_size, padding, stride, dilation):
        super(ConvStridedBlock, self).__init__()
        
        self.cnn1 = nn.Conv2d(in_channels=input_size, out_channels=out_channels, kernel_size=kernel_size, padding=padding, stride=stride, dilation=1, bias=False)
        self.batchNorm = nn.BatchNorm2d(out_channels)
        self.cnn2 = nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=kernel_size, padding=1, stride=stride, dilation=1, bias=False)
        self.dilated = nn.Conv2d(in_channels=input_size, out_channels=out_channels, kernel_size=kernel_size, padding=1, stride=stride, dilation=dilation, bias=False)
        self.dropout = nn.Dropout(0.2)
        self.gelu = nn.GELU()
        self.leaky = nn.LeakyReLU()
        
    def forward(self, x):
        residual = x
        x = self.cnn1(x)
        x = self.batchNorm(x)
#         x = self.leaky(x)
        
        x = self.cnn2(x)
        x = self.batchNorm(x)
#         x = self.leaky(x)
        
        residual = self.dilated(residual)
        x = x + residual
        x = self.leaky(x)
        
        return x

In [ ]:
csb = ConvStridedBlock(3, 32, 3, 0, 1, 2)

summary(model=csb,
        input_size=(1, 3, 512, 512), # (batch_size, num_patches, embedding_dimension)
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

In [ ]:
model = CNN()

summary(model=model,
        input_size=(1, 3, 512, 512), # (batch_size, num_patches, embedding_dimension)
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

### Vision Transformer

In [ ]:
PATCH_SIZE      = 16
IMAGE_WIDTH     = image_size[0]
IMAGE_HEIGHT    = image_size[1]
IMAGE_CHANNELS  = 3
EMBEDDING_DIMS  = IMAGE_CHANNELS * PATCH_SIZE**2 # 768
NUM_OF_PATCHES  = int((IMAGE_WIDTH * IMAGE_HEIGHT) / PATCH_SIZE**2) # 256
DEVICE          = torch.device("cuda" if torch.cuda.is_available() else "cpu")

assert IMAGE_WIDTH % PATCH_SIZE == 0 and IMAGE_HEIGHT % PATCH_SIZE ==0 , print("Image Width is not divisible by patch size")

In [ ]:
print(EMBEDDING_DIMS)
print(NUM_OF_PATCHES)

In [ ]:
class MachineLearningPerceptronBlock(nn.Module):
    def __init__(self, embedding_dims, mlp_size, mlp_dropout):
        super().__init__()
        self.embedding_dims = embedding_dims
        self.mlp_size = mlp_size
        self.dropout = mlp_dropout

        self.layernorm = nn.LayerNorm(normalized_shape = embedding_dims)
        self.mlp = nn.Sequential(
            nn.Linear(in_features = embedding_dims, out_features = mlp_size),
            nn.GELU(),
            nn.Dropout(p = mlp_dropout),
            nn.Linear(in_features = mlp_size, out_features = embedding_dims),
            nn.Dropout(p = mlp_dropout)
        )

    def forward(self, x):
        return self.mlp(self.layernorm(x))

class TransformerEncoder(nn.Module):
    def __init__(self,
                 embedding_dims = EMBEDDING_DIMS, # Hidden Size D in the ViT Paper Table 1
                 num_heads = 8,  # Heads in the ViT Paper Table 1
                 attn_dropout = 0.0,
                 mlp_size = 3072,
                 mlp_dropout = 0.1): # Default to Zero as there is no dropout for the the MSA Block as per the ViT Paper):
        super(TransformerEncoder, self).__init__()
                 
        self.layerNorm = nn.LayerNorm(EMBEDDING_DIMS)
        self.multiheadattention = nn.MultiheadAttention(num_heads = num_heads, 
                                               embed_dim = embedding_dims, 
                                               dropout = attn_dropout, 
                                               batch_first = True)
        self.mlp = MachineLearningPerceptronBlock(embedding_dims = embedding_dims, 
                                                  mlp_size = mlp_size,
                                                  mlp_dropout = mlp_dropout)
        
    def forward(self, x):
        x2 = self.layerNorm(x)
        output, _ = self.multiheadattention(query=x2, key=x2, value=x2, need_weights=False)
        
        x = x + x2
        
        x2 = self.layerNorm(x)
        x2 = self.mlp(x2)
        
        x = x + x2
        return x
    
    
class SampleViT(nn.Module):
    def __init__(self, img_size = IMAGE_HEIGHT,
                 in_channels = IMAGE_CHANNELS,
                 patch_size = PATCH_SIZE,
                 embedding_dims = EMBEDDING_DIMS,
                 num_transformer_layers = 8, # from table 1 above
                 mlp_dropout = 0.1,
                 attn_dropout = 0.0,
                 mlp_size = 3072,
                 num_heads = 12,
                 num_classes = 1):
        super(SampleViT, self).__init__()
        
        self.positional_embedding = nn.Parameter(torch.randn(1, NUM_OF_PATCHES + 1, EMBEDDING_DIMS))
        self.class_token = nn.Parameter(torch.rand((1, 1, EMBEDDING_DIMS), requires_grad=True))

        self.patchConv = conv_layer = nn.Conv2d(in_channels = IMAGE_CHANNELS, out_channels = EMBEDDING_DIMS, kernel_size = PATCH_SIZE, stride = PATCH_SIZE)
        self.transformer_encoder = nn.Sequential(*[TransformerEncoder(embedding_dims = embedding_dims,
                                              mlp_dropout = mlp_dropout,
                                              attn_dropout = attn_dropout,
                                              mlp_size = mlp_size,
                                              num_heads = num_heads) for _ in range(num_transformer_layers)])

        self.classifier = nn.Sequential(nn.LayerNorm(normalized_shape = embedding_dims),
                                    nn.Linear(in_features = embedding_dims,
                                              out_features = num_classes))
    
    def patch(self, images):
        batch_size, c, h, w = images.shape
        assert h == w # Square image only
            
        patch = self.patchConv(images)
        patch = patch.view(batch_size, patch.shape[1], -1).permute(0, 2, 1)
        
        return patch
    
    def forward(self, x):
        patch = self.patch(x)
#         class_token = nn.Parameter(torch.rand((x.size(0), 1,EMBEDDING_DIMS), requires_grad  = True)).to(DEVICE)
        class_token = self.class_token.expand(x.size(0), -1, -1)  # Expand class token for each element in the batch
        embedded_image_with_class_token_embeddings = torch.cat((class_token, patch), dim=1)
        embedded_patch = embedded_image_with_class_token_embeddings + self.positional_embedding
        return torch.sigmoid(self.classifier(self.transformer_encoder(embedded_patch)[:, 0]))


In [ ]:
model = SampleViT().to(DEVICE)

In [ ]:
mlp = MachineLearningPerceptronBlock(embedding_dims = EMBEDDING_DIMS, 
                                     mlp_size = 3072,
                                     mlp_dropout = 0.1)

summary(model=mlp,
        input_size=(5, 256, 768), # (batch_size, num_patches, embedding_dimension)
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

In [ ]:
te = TransformerEncoder()

summary(model=te,
        input_size=(5, 256, 768), # (batch_size, num_patches, embedding_dimension)
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

In [ ]:
summary(model=model,
        input_size=(1, 3, 256, 256), # (batch_size, num_patches, embedding_dimension)
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,)

# Training Loop

In [ ]:
from tqdm import tqdm


criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.00001, weight_decay=1e-4)
num_epochs = 20

# Training loop
def train_model(model, train_loader, criterion, optimizer, num_epochs=20):
    model.to(DEVICE)
    
    for epoch in range(num_epochs):
        model.train()
        loop = tqdm(train_loader, leave=True, desc=f'Epoch {epoch + 1}/{num_epochs}')

        total_loss = 0
        correct_predictions = 0
        total_samples = 0

        for batch_idx, (batch_inputs, batch_labels) in enumerate(loop):
            batch_inputs, batch_labels = batch_inputs.to(DEVICE), batch_labels.to(DEVICE)
            optimizer.zero_grad()

            # Forward pass
            outputs = model(batch_inputs)

            # Compute loss
            loss = criterion(outputs.squeeze(), batch_labels.float())

            # Backward pass
            loss.backward()

            # Update weights
            optimizer.step()
#             loop.set_postfix(
#                 loss=loss.item(),
#             )

            total_loss += loss.item()
            predicted_labels = torch.round(outputs)  # Assuming sigmoid activation for binary classification
            correct_predictions += (predicted_labels == batch_labels).sum().item()
            total_samples += batch_labels.size(0)

            # Update tqdm bar description with the current loss and accuracy
            loop.set_postfix(loss=total_loss / (batch_idx + 1), accuracy=correct_predictions / total_samples)

        accuracy = correct_predictions / total_samples
        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader)}, Accuracy: {accuracy}%')

In [ ]:
train_model(model, train_loader, criterion, optimizer, num_epochs)